# Моделирование данных 1

Машинное обучение и наука о данных - это моделирование данных. **Моделирование** - это представление идеи с некоторыми параметрами и математическое представление, которое мы будем кодировать в программном обеспечении. Все методы машинного обучения направлены на то, чтобы научить компьютер приспосабливать модель к некоторым данным. Даже самые модные нейронные сети - просто выбор моделей. В этом блокноте мы начнем строить нашу первую вычислительную модель данных.

## Моделирование данных это сложно!

Давайте найдем, где мы остановились в блокноте 1 с фруктами. У нас осталась загадка: когда мы загружаем изображения яблок и бананов,

In [ ]:
using Images 

In [ ]:
apple = load("D:/Science/data/10_100.jpg")

In [ ]:
banana = load("D:/Science/data/104_100.jpg")

и затем сравниваем их среднее значение для красного цвета, в результате получается нечто удивительное:

In [ ]:
apple_red_amount = mean(Float64.(red.(apple)))
banana_red_amount = mean(Float64.(red.(banana)));

"Среднее значение красного в яблоке $apple_red_amount, " *
"в то время как среднее значение красного в банане $banana_red_amount."

Мы видим, что среднее значение красного банана выше, чем у яблока, хотя яблоко выглядит намного краснее. Вы можете догадаться, почему? 

На самом деле есть две причины. Одной из причин является фон: изображение банана имеет намного больше белого фона, чем яблоко, и этот белый фон имеет красное значение = 1! Мы не обращаем внимания на фон и говорим: «банан ярко-желтый, яблоко темно-красное», но у компьютера просто есть набор цифр, и он не знает, где его искать. 

Другая проблема заключается в том, что «ярко-желтый» - это не тот цвет, который существует в компьютере. Компьютер имеет три цвета: красный, зеленый и синий. «Ярко-желтый» в компьютере - это смесь красного и зеленого, и так получилось, что для того, чтобы получить желтый цвет, ему нужно больше красного, чем для яблока!

In [ ]:
"Количество красного в яблоке на (60, 60) - $(Float64(red(apple[60, 60]))), " *
"в то время как количество красного в банане в (60, 60) будет $(Float64(red(banana[60, 60])))."

In [ ]:
apple[60,60]

In [ ]:
banana[60,60]

Это яркий пример того, что моделировать данные сложно!

### Примечание о интерполяции строк

В последних двух входных ячейках мы *интерполировали строку*. Это означает, что когда мы пишем строку, используя кавычки (`" "`), мы вставляем заполнитель для некоторого **значения**, которое мы хотим, чтобы строка включала. Когда строка оценивается, значение, которое мы хотим включить в строку, заменяет заполнитель. Например, в следующей строке

```julia
mystring = "Среднее значение красного в яблоке $apple_red_amount"
```

`$apple_red_amount` является заполнителем для значения, хранящегося в переменной `apple_red_amount`. Юлия знает, что мы хотим использовать значение, связанное с переменной `apple_red_amount` а *не* слово "apple_red_amount" из-за знака доллара, `$`, что стоит перед `apple_red_amount`.

#### Упражнение 1

Выполните следующий код, чтобы увидеть, что делает знак доллара:

```julia
mypi = 3.14159
println("У меня есть переменная с именем mypi, которая имеет значение $mypi.")
```

#### Упражнение 2

Измените и выполните код, который создает `mystring` ниже 

```julia
apple_blue_amount = mean(Float64.(blue.(apple)))
mystring = "Среднее количество синего в яблоке apple_blue_amount"
```

так чтоб `println (mystring)` печатал строку, которая сообщает среднее значение синей окраски в нашем изображении яблока.

## Поразмыслим о данных

Яблоки и бананы очень разные, но как мы можем использовать массив значений RGB (как изображения представлены в компьютере мы видели в блокноте 1), чтобы определить разницу между ними? Вот что первое приходит на ум (~~но не мне~~):

- Мы могли бы использовать форму объекта на изображении. Но как мы можем кодировать идеи о форме из массива?
- Мы могли бы использовать размер объекта на изображении. Но как мы можем рассчитать этот размер?
- Мы могли бы использовать другой цвет или комбинации цветов из изображения. Какие цвета?

Попробуем последнее. Банан желтый, который является комбинацией красного и зеленого, а яблоко красное. Это означает, что цвет, который четко отличается от этих двух цветов зеленый!

In [ ]:
apple_green_amount = mean(Float64.(green.(apple)))
banana_green_amount = mean(Float64.(green.(banana)));

"Среднее значение зеленого в яблоке $apple_green_amount, " *
"в то время как среднее значение зеленого в банане $banana_green_amount."

Процессам, которые мы только что испробовали, присваиваются причудливые имена: выбор функций и обработка данных. 

**Выбор функции** - это процесс подстановки данных в более релевантный и информативный набор. Мы взяли полные данные изображения и решили выбрать зеленый канал. 

**Обработка данных** преобразует данные в формат, более подходящий для моделирования. Здесь, вместо сохранения полного зеленого канала, мы преобразовали его в одну точку данных: среднее количество зеленых.

## Построение модели

Мы хотим смоделировать связь между «средним количеством зеленого» и «яблоком или бананом».

<img src="data/data_flow.png" alt="Drawing" style="width: 800px;"/>

Эта модель представляет собой математическую функцию, которая принимает наши данные и выдает число, которое мы будем интерпретировать как «это яблоко» или «это банан».

<img src="data/what_is_model.png" alt="Drawing" style="width: 500px;"/>


Мы будем интерпретировать выходные данные функции как «is apple», если выходные данные близки к 0, и «is banana», если они близки к 1. Что-то в середине - это то, в чем мы не уверены. 

Здесь мы используем математическую функцию для выполнения **классификации**. Знание того, как объявлять и работать с функциями, позволит нам смоделировать наши данные в следующих разделах, так что это тема следующего блокнота!